In [1]:
import tensorflow.compat.v1 as tf
import lucid_kietzmannlab.modelzoo.vision_models as models
import matplotlib.pyplot as plt
from tqdm import tqdm
import lucid_kietzmannlab.optvis.objectives as objectives
import lucid_kietzmannlab.optvis.render as render
from ipywidgets import interact, Dropdown, IntSlider
from IPython.display import clear_output
from lucid_kietzmannlab.modelzoo.vision_models import get_layer_names_tensors
from lucid_kietzmannlab.recipes.image_activations import image_activations
tf.compat.v1.disable_eager_execution()

In [2]:
model_path = '/Users/vkapoor/Downloads/models/AlexNet/seed5.pb'
model = models.AlexNetv2(model_path=model_path)


/Users/vkapoor/miniconda3/envs/lucidenv/lib/python3.12/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
2024-06-09 14:35:19.434305: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


In [3]:
layer_shape_dict = model.layer_shape_dict
C = lambda neuron: objectives.channel(*neuron)    
def visualize(layer_name, channel):
    # Check if the layer exists in the shape dictionary
    if layer_name in layer_shape_dict:
        # Check if the selected channel is within bounds
        max_channel = layer_shape_dict[layer_name][-1] - 1
        if 0 <= channel <= max_channel:
            clear_output(wait=True)
            # Render visualization for the selected layer and channel
            try:
              _ = render.render_vis(model, C((layer_name, channel)), scope = 'alexnet_v2')
            except Exception:
                print('No gradients for this layer')   

def visualize_all():
    # Check if the layer exists in the shape dictionary
    layer_name = current_dropdown_value({'new': layer_dropdown.value})
    print(layer_name)
    if layer_name in layer_shape_dict:
            # Check if the selected channel is within bounds
            try:
               image_channel = {} 
               for channel in tqdm(range(channel_slider.max)):    
                  images = render.render_vis(model, C((layer_name, channel)), verbose = False, scope = 'alexnet_v2')
                  image_channel[channel] = images 
            except Exception:
                print('No gradients for this layer')    
    return image_channel            
                              
def plot_images(image_channel):
    channels = list(image_channel.keys())
    for i in range(0, len(channels), 3):
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))  
        for j in range(3):
            channel_index = i + j
            if channel_index < len(channels):
                channel = channels[channel_index]
                images = image_channel[channel]
                try:
                    axs[j].imshow(images[0][0,:])  
                    axs[j].set_title(f"Channel {channel}")
                    axs[j].axis('off')
                except Exception as e:
                    print(f"Error plotting channel {channel}: {e}")
                    axs[j].axis('off')
                    axs[j].text(0.5, 0.5, 'Error', ha='center', va='center')
            else:
                axs[j].axis('off')  
        plt.tight_layout()
        plt.show()
        
# Create dropdown menu for layer selection
layer_dropdown = Dropdown(options=list(layer_shape_dict.keys()), description='Layer:')

# Create slider for channel selection
channel_slider = IntSlider(min=0, max=0, description='Channel:')
        
        
def update_channel_slider(change):
    layer_name = change.new
    if layer_name in layer_shape_dict:
        
        max_channel = layer_shape_dict[layer_name][-1] - 1
        channel_slider.max = max_channel
        
        
def current_slider_value(*args):
    return channel_slider.value


def current_dropdown_value(change):
    return change['new']


channel_slider.observe(current_slider_value, names='value')
layer_dropdown.observe(current_dropdown_value, names='value')
        

       

In [4]:
layer_dropdown.observe(update_channel_slider, names='value')

# Create an interactive visualization
interact(visualize, layer_name=layer_dropdown, channel=channel_slider)     

interactive(children=(Dropdown(description='Layer:', options=('conv1/Conv2D', 'conv2/Conv2D', 'conv3/Conv2D', …

<function __main__.visualize(layer_name, channel)>

In [ ]:
image_channel = visualize_all()

In [ ]:

if image_channel:
    plot_images(image_channel)   

In [ ]:
layer_shape_dict.values()